In [1]:
# IMPORT PYTHON MODULES
import pandas as pd
import numpy as np

In [2]:
# IMPORT FOUR ACS DATASETS
housing = pd.read_csv('Housing.csv')
social = pd.read_csv('Social.csv')
econ = pd.read_csv('Economic.csv')
demo = pd.read_csv('Demographics.csv')

In [3]:
housing.set_index('GEO_ID', inplace=True)
social.set_index('GEO_ID', inplace=True)
econ.set_index('GEO_ID', inplace=True)
demo.set_index('GEO_ID', inplace=True)

In [4]:
# MERGE ACS DATASETS
complete = pd.concat([housing, social, econ, demo], axis=1, join="inner")

In [5]:
# SELECT RELEVANT VARIABLES
filtered = complete[['DP03_0128PE', 'DP02_0067PE', 'DP05_0032PE', 'DP05_0072PE', 'DP05_0033PE', 'DP05_0073PE', 'DP05_0039PE',
                   'DP05_0075PE', 'DP05_0034PE', 'DP05_0074PE', 'DP05_0047PE', 'DP05_0076PE', 'DP05_0052PE', 'DP05_0077PE', 
                   'DP05_0053PE', 'DP05_0078PE', 'DP04_0025PE', 'DP04_0024PE', 'DP04_0003PE', 'DP04_0045PE', 'DP02_0003PE']]

In [6]:
new_cols = ['Poverty', 'Bachelor', 'White', 'White-NH', 'Black', 'Black-NH', 'Asian', 'Asian-NH', 'AmericanIndian', 
            'AmericanIndian-NH', 'NHPI', 'NHPI-NH', 'Other', 'Other-NH', 'Multi', 'Multi-NH', 'Pre-1939 Housing', 
           '1939-1949 Housing', 'VacantHousing', 'OwnerOccupied', 'FamiliesWithChildren']
filtered.columns = new_cols
filtered.drop(['id'], inplace=True)

C:\Users\Srivinay Tummarakota\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [7]:
# CONVERT COLUMN DATA TYPES TO NUMERIC
for col in filtered.columns:
    filtered.drop(filtered[filtered[col]=='-'].index, inplace=True)
    filtered.loc[:, col] = pd.to_numeric(filtered.loc[:, col])

C:\Users\Srivinay Tummarakota\anaconda3\lib\site-packages\pandas\core\indexing.py:1047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
C:\Users\Srivinay Tummarakota\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [8]:
# CALCULATE % OF RACE THAT IS NON-HISPANIC
races = ['White', 'Black', 'Asian', 'AmericanIndian', 'NHPI', 'Other', 'Multi']
for race in races:
    filtered.loc[:, race+'-H'] = filtered.loc[:, race]-filtered.loc[:, race+'-NH']

C:\Users\Srivinay Tummarakota\anaconda3\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


In [9]:
# CALCULATE SIMPSON'S DIVERSITY INDEX
races_ethnicities = [race+'-H' for race in races]+[race+'-NH' for race in races]
pi = [filtered.loc[:, race]/100 for race in races_ethnicities]
pi_complement = [1-col for col in pi]
filtered.loc[:, 'SimpsonsDI'] = sum([pi[i]*pi_complement[i] for i in range(len(pi))])

In [10]:
# CHANGE INDEX TO CENSUS TRACT NUMBER
filtered.set_index(filtered.index.str[9:], inplace=True)

In [11]:
# CALCULATE OLD-HOUSING AS PERCENTAGE OF ALL HOUSING BUILT PRIOR TO 1950
filtered.loc[:, 'OldHousing'] = filtered.loc[:, 'Pre-1939 Housing'] + filtered.loc[:, '1939-1949 Housing']

In [12]:
filtered.to_csv('filtered_data.csv')

# Merge Lead and ACS Datasets

In [13]:
lead = pd.read_csv('lead.csv')

In [14]:
lead.dropna(how='any', inplace=True)

In [15]:
lead.columns = ['GEO_ID', 'IsRedacted', 'NumBll5Plus', 'NumScreen', 'Pct5Plus']
lead.set_index('GEO_ID', inplace=True)
lead.index = lead.index.map(str)

In [19]:
# MERGE ACS AND LEAD DATASETS
merged = pd.concat([filtered, lead], axis=1, join="inner")
merged = merged[['Bachelor', 'Poverty', 'OldHousing', 'Black', 'SimpsonsDI', 'VacantHousing', 'OwnerOccupied', 'FamiliesWithChildren', 'Pct5Plus']]

In [20]:
merged.to_csv('cleaned_data.csv')

In [21]:
merged.head()

,Bachelor,Poverty,OldHousing,Black,SimpsonsDI,VacantHousing,OwnerOccupied,FamiliesWithChildren,Pct5Plus
GEO_ID,,,,,,,,,
42101000100,79.8,11.7,65.7,10.5,0.438901,16.2,29.3,3.6,0.0
42101000402,74.5,10.7,19.3,2.0,0.212096,13.0,61.9,1.0,0.0
42101000600,80.0,22.5,83.9,10.3,0.475657,12.8,2.0,2.3,0.0
42101000700,77.5,28.8,53.0,11.0,0.587736,18.4,24.8,2.8,0.0
42101000803,85.3,14.9,36.0,0.5,0.216456,9.7,39.2,3.5,0.0
